In [1]:
from random import random, randint, seed
from tqdm import tqdm
from __future__ import division

In [2]:
# seed(41)

In [3]:
class MontyHall(object):
    '''
    Plays the game posited in the classic Monty Hall problem:
    https://en.wikipedia.org/wiki/Monty_Hall_problem
    '''
    def __init__(self):
        self.player_pick = randint(0,2)
        self.prize_pick = randint(0,2)
        self.switch = self.switch_decision()
        self.host_reveal = self.host_reveal()
        self.final_pick = self.make_final_pick()
        self.match = self.was_match()

    def switch_decision(self):
        return random() >= .5
    
    def host_reveal(self):
        doors = [0,1,2]
        if self.prize_pick == self.player_pick:
            doors.remove(self.prize_pick)
            return doors[0] if random() >= .5 else doors[1]
        else:
            doors.remove(self.player_pick)
            doors.remove(self.prize_pick)
            return doors[0]
    
    def make_final_pick(self):
        if self.switch:
            doors = [0,1,2]
            doors.remove(self.host_reveal)
            doors.remove(self.player_pick)
            return doors[0]
        else:
            return self.player_pick
    
    def was_match(self):
        return self.final_pick == self.prize_pick
    
    def __str__(self):
        string = 'Player picked Door {}'.format(self.player_pick)
        string += '\nPrize behind Door {}'.format(self.prize_pick)
        string += '\nHost reveals Door {}'.format(self.host_reveal)
        string += '\nPlayer decides to {}'.format('SWITCH' if self.switch else 'STAY')
        string += '\nPlayer final pick is {}'.format(self.final_pick)
        string += '\nThere {} a match'.format('WAS' if self.match else 'WAS NOT')
        return string

In [4]:
games = [MontyHall() for _ in range(0,100000)]

In [5]:
switch_match = sum([g.switch and g.match for g in games])
switch_nomatch = sum([g.switch and not g.match for g in games])
noswitch_match = sum([not g.switch and g.match for g in games])
noswitch_nomatch = sum([not g.switch and not g.match for g in games])

In [6]:
print "By switching, you have a {}% chance of winning".format(
     switch_match / (switch_match+switch_nomatch) * 100
)

By switching, you have a 66.6646692234% chance of winning


In [7]:
print "By NOT switching, you have a {}% chance of winning".format(
     noswitch_match / (noswitch_match+noswitch_nomatch) * 100
)

By NOT switching, you have a 33.1924863826% chance of winning


In [8]:
def resample_once(games=100):
    # play n games
    games = [MontyHall() for _ in range(0,games)]
    
    # split games up into switch and stay groups (should be about evenly split)
    switches = [g for g in games if g.switch]
    stays = [g for g in games if not g.switch]
    
    # resample from each with random integers
    resampled_switches = [switches[randint(0,len(switches)-1)] for _ in switches]
    resampled_stays = [stays[randint(0,len(stays)-1)] for _ in stays]
    
    # return True if switching resulted in a match more often than staying
    switch_match_pct = sum([g.match for g in resampled_switches]) / len(resampled_switches)
    stay_match_pct = sum([g.match for g in resampled_stays]) / len(resampled_stays)
    return switch_match_pct > stay_match_pct

In [9]:
resample_n_times = lambda x: [resample_once() for _ in tqdm(range(0,x))]

In [10]:
result = resample_n_times(100000)

100%|██████████| 100000/100000 [01:27<00:00, 1137.31it/s]


In [11]:
print 'Switching was the better choice {}% of the time'.format(sum(result)/len(result)*100)

Switching was the better choice 99.193% of the time


In [12]:
print 'This implies a p-value of {}'.format(1-sum(result)/len(result))

This implies a p-value of 0.00807
